In [48]:
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import functions
from pyspark.sql.window import Window

In [3]:
parquet_dataset="/home/hari/Downloads/LearningSparkV2-master/parquet_files/userdata1.parquet"

    Main method definition and spark object creation

In [4]:
if __name__ == "__main__" :
    spark=(SparkSession
           .builder
           .appName("PrPg2_DF_manipulation")
           .getOrCreate()
          )

23/05/01 09:19:34 WARN Utils: Your hostname, hps17 resolves to a loopback address: 127.0.1.1; using 192.168.236.132 instead (on interface ens33)
23/05/01 09:19:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/01 09:19:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
parq_DF=spark.read.format("parquet").load(parquet_dataset)

In [6]:
parq_DF.show()
parq_DF.printSchema()

+-------------------+---+----------+---------+--------------------+------+---------------+-------------------+--------------------+----------+---------+--------------------+--------------------+
|  registration_dttm| id|first_name|last_name|               email|gender|     ip_address|                 cc|             country| birthdate|   salary|               title|            comments|
+-------------------+---+----------+---------+--------------------+------+---------------+-------------------+--------------------+----------+---------+--------------------+--------------------+
|2016-02-03 13:25:29|  1|    Amanda|   Jordan|    ajordan0@com.com|Female|    1.197.201.2|   6759521864920116|           Indonesia|  3/8/1971| 49756.53|    Internal Auditor|               1E+02|
|2016-02-03 22:34:03|  2|    Albert|  Freeman|     afreeman1@is.gd|  Male| 218.111.175.34|                   |              Canada| 1/16/1968|150280.17|       Accountant IV|                    |
|2016-02-03 06:39:31|  3|

Scenario:
1. use select function to fetch selectiv column from DF
2. Where clause to limit the records/data
3. filter vs where function usage
4.

In [18]:
(parq_DF.select("country","title","gender")
 .where(trim(col("title")) == '')).show()

+--------------------+-----+------+
|             country|title|gender|
+--------------------+-----+------+
|        South Africa|     |      |
|              Russia|     |Female|
|              Brazil|     |  Male|
|              Russia|     |  Male|
|               China|     |Female|
|            Tanzania|     |  Male|
|              Russia|     |Female|
|       United States|     |Female|
|            Thailand|     |  Male|
|       United States|     |Female|
|              France|     |Female|
|               China|     |Female|
|Central African R...|     |  Male|
|            Ethiopia|     |Female|
|               China|     |  Male|
|              Brazil|     |Female|
|           Indonesia|     |  Male|
|               China|     |Female|
|              France|     |  Male|
|           Indonesia|     |Female|
+--------------------+-----+------+
only showing top 20 rows



# DF manipulation

> when & otherwise (equivalent to If - else conditional block)
> trim
> withColumn functions
> drop ->> to dorp column in DF
> withColumnRenamed
> filter & where - both produce the samee output
> groupBy(), count() and orderBy()window function
> WINDOW functions:
    > rank()
    > dense_rank()
    > row_numberr()
    > percent_rank()
    > ntile()
    
    

In [59]:
DF1.printSchema()
win=Window.partitionBy("title_clnd").orderBy("country")

DF1.withColumn("ID",row_number().over(win)).show()
DF1.withColumn("Rank_ID",rank().over(win)).show()
DF1.withColumn("percent_Rank",percent_rank().over(win)).show()
DF1.withColumn("ntile",ntile(3).over(win)).show()

root
 |-- title_clnd: string (nullable = true)
 |-- country: string (nullable = true)
 |-- Gender_Orientation: string (nullable = true)



+--------------------+----------------+------------------+---+
|          title_clnd|         country|Gender_Orientation| ID|
+--------------------+----------------+------------------+---+
|            61871.37|        "Bonaire|              Male|  1|
| Account Coordinator|         Finland|            Female|  1|
| Account Coordinator|           Japan|              Male|  2|
| Account Coordinator|         Nigeria|              Male|  3|
| Account Coordinator|     Philippines|            Female|  4|
|   Account Executive|   Aland Islands|              Male|  1|
|   Account Executive|          Canada|              Male|  2|
|   Account Executive|           China|            Female|  3|
|   Account Executive|           China|            Female|  4|
|   Account Executive|          Guinea|            Female|  5|
|   Account Executive|       Indonesia|            Female|  6|
|   Account Executive|       Indonesia|              Male|  7|
|   Account Executive|Papua New Guinea|              Ma

+--------------------+----------------+------------------+-------+
|          title_clnd|         country|Gender_Orientation|Rank_ID|
+--------------------+----------------+------------------+-------+
|            61871.37|        "Bonaire|              Male|      1|
| Account Coordinator|         Finland|            Female|      1|
| Account Coordinator|           Japan|              Male|      2|
| Account Coordinator|         Nigeria|              Male|      3|
| Account Coordinator|     Philippines|            Female|      4|
|   Account Executive|   Aland Islands|              Male|      1|
|   Account Executive|          Canada|              Male|      2|
|   Account Executive|           China|            Female|      3|
|   Account Executive|           China|            Female|      3|
|   Account Executive|          Guinea|            Female|      5|
|   Account Executive|       Indonesia|            Female|      6|
|   Account Executive|       Indonesia|              Male|    

+--------------------+----------------+------------------+-----+
|          title_clnd|         country|Gender_Orientation|ntile|
+--------------------+----------------+------------------+-----+
|            61871.37|        "Bonaire|              Male|    1|
| Account Coordinator|         Finland|            Female|    1|
| Account Coordinator|           Japan|              Male|    1|
| Account Coordinator|         Nigeria|              Male|    2|
| Account Coordinator|     Philippines|            Female|    3|
|   Account Executive|   Aland Islands|              Male|    1|
|   Account Executive|          Canada|              Male|    1|
|   Account Executive|           China|            Female|    1|
|   Account Executive|           China|            Female|    2|
|   Account Executive|          Guinea|            Female|    2|
|   Account Executive|       Indonesia|            Female|    2|
|   Account Executive|       Indonesia|              Male|    3|
|   Account Executive|Pap

In [45]:
DF1=(parq_DF.withColumn("title_clnd", when(trim(col("title")) == '','TBD')
       .otherwise(parq_DF.title))
       .withColumnRenamed("gender","Gender_Orientation")
       .filter((col("Gender_Orientation") == 'Male') | (col("Gender_Orientation") == 'Female'))
       .drop("title")
       .select("title_clnd","country","Gender_Orientation")
)
DF1.printSchema()
DF2=(DF1.groupBy("title_clnd","country","Gender_Orientation").count()
 .orderBy("country",asc="true")   
)
DF2.show()

root
 |-- title_clnd: string (nullable = true)
 |-- country: string (nullable = true)
 |-- Gender_Orientation: string (nullable = true)

+--------------------+-------------+------------------+-----+
|          title_clnd|      country|Gender_Orientation|count|
+--------------------+-------------+------------------+-----+
|            61871.37|     "Bonaire|              Male|    1|
|                 TBD|  Afghanistan|              Male|    1|
|                 TBD|  Afghanistan|            Female|    1|
|   Account Executive|Aland Islands|              Male|    1|
|                 TBD|      Albania|            Female|    1|
|           Professor|      Albania|            Female|    1|
|  Research Associate|      Albania|            Female|    1|
|                 TBD|       Angola|            Female|    1|
|                 TBD|    Argentina|              Male|    1|
|   Marketing Manager|    Argentina|              Male|    1|
|   Chemical Engineer|    Argentina|            Female|  